# MGSC662 Group Project

## Initial setup

In [1]:
import pandas as pd
import gurobipy as gp
from gurobipy import *
import numpy as np

In [2]:
# Import demand data
demand_blue = pd.read_excel('Demand_estimation_Index_Dec1.xlsx', sheet_name='blue', index_col = 0)
demand_orange = pd.read_excel('Demand_estimation_Index_Dec1.xlsx', sheet_name='orange', index_col = 0)
demand_yellow = pd.read_excel('Demand_estimation_Index_Dec1.xlsx', sheet_name='yellow', index_col = 0)
demand_green = pd.read_excel('Demand_estimation_Index_Dec1.xlsx', sheet_name= 'green', index_col = 0)

# Change to array
arr_demand_blue = demand_blue.to_numpy()
arr_demand_orange = demand_orange.to_numpy()
arr_demand_yellow = demand_yellow.to_numpy()
arr_demand_green = demand_green.to_numpy()

# Length
num_hour_period = demand_blue.shape[0] # i = 20
num_day_type = demand_blue.shape[1] # j = 4

In [3]:
# View the demand of green line, for instance
demand_green

,Weekday,Friday,Saturday,Sunday
1,8890,8890,8890,8890
2,20003,20003,8890,8890
3,20003,20003,13336,13336
4,16669,16669,16669,16669
5,13336,13336,20003,20003
6,13336,13336,13336,13336
7,13336,13336,13336,13336
8,13336,13336,13336,13336
9,13336,13336,13336,13336
10,13336,13336,13336,13336


In [4]:
# Import general data
num_station = 68
num_line = 4

In [5]:
# Import capacity of each car
mr73_car = 160
mpm10_car = 172

mr73_seats_car = 40
mpm10_seats_car = 32

In [6]:
# Import number of available trains
orange_mpm10 = 45
green_mpm10 = 36
green_mr73 = 6
blue_mr73 = 18
yellow_mr73 = 5

# Import number of cars per train
num_cars = [6, 9, 9, 9] # The order is: Blue, Orange, Yellow, Green

In [7]:
# Import maximum average waiting time
# The order is: Blue, Orange, Yellow, Green
mon_fri_peak = [5, 2.5, 5, 5] 
mon_fri_nonpeak = [10, 10, 10, 10]
sat_sun = [11, 12, 10, 12]

In [8]:
# Fixed cost and variable cost
fixed_cost = 1877769 # fixed cost

# Weighted average variable cost per departure per line
variable_cost = [651, 1519, 180, 1461] # The order is: Blue, Orange, Yellow, Green

# Weekly budget for overall operation (maximum cost allowance)
budget = 6453730

# Define name
hour = ['5-6am', '6-7am', '7-8am', '8-9am', '9-10am', '10-11am', '11am-12pm', 
        '12-13pm', '13-14pm', '14-15pm', '15-16pm', '16-17pm', '17-18pm', '18-19pm', 
       '19-20pm', '20-21pm', '21-22pm', '22-23pm', '23pm-12am', '12am-1am']
time = ['Mon-Thu', 'Fri', 'Sat', 'Sun']

## Minimizing cost model

In [9]:
# Create model
model = gp.Model('STM minimizing cost model')

Academic license - for non-commercial use only - expires 2022-09-25
Using license file /Users/AngelaChen/gurobi.lic


In [10]:
# Create decision variables
x_blue = model.addVars(num_hour_period, num_day_type, vtype = GRB.INTEGER, 
                       name = ['Blue #departure ' + str(i) + '_' + str(j) 
                               for i in hour for j in time])

x_orange = model.addVars(num_hour_period, num_day_type, vtype = GRB.INTEGER, 
                         name = ['Orange #departure ' + str(i) + '_' + str(j) 
                               for i in hour for j in time])

x_yellow = model.addVars(num_hour_period, num_day_type, vtype = GRB.INTEGER, 
                         name = ['Yellow #departure ' + str(i) + '_' + str(j) 
                               for i in hour for j in time])

x_green = model.addVars(num_hour_period, num_day_type, vtype = GRB.INTEGER, 
                        name = ['Green #departure ' + str(i) + '_' + str(j) 
                               for i in hour for j in time])

In [11]:
# Objective function
# Blue
obj1 = 4 * sum(x_blue[i, 0] * variable_cost[0]
               for i in range(num_hour_period)) + sum(x_blue[i, j] * variable_cost[0]
                                                      for i in range(num_hour_period) 
                                                      for j in range(1, num_day_type))
# Orange
obj2 = 4 * sum(x_orange[i, 0] * variable_cost[1]
               for i in range(num_hour_period)) + sum(x_orange[i, j] * variable_cost[1]
                                                      for i in range(num_hour_period) 
                                                      for j in range(1, num_day_type))

# Yellow
obj3 = 4 * sum(x_yellow[i, 0] * variable_cost[2]
               for i in range(num_hour_period)) + sum(x_yellow[i, j] * variable_cost[2]
                                                      for i in range(num_hour_period) 
                                                      for j in range(1, num_day_type))

# Green
obj4 = 4 * sum(x_green[i, 0] * variable_cost[3]
               for i in range(num_hour_period)) + sum(x_green[i, j] * variable_cost[3]
                                                      for i in range(num_hour_period) 
                                                      for j in range(1, num_day_type))
# Set objective function
model.setObjective(obj1 + obj2 + obj3 + obj4 + fixed_cost, GRB.MINIMIZE)

In [12]:
# Constraints
# 1. Capacity > Demand
# Notice that the train capacity is halved because of social distancing
for i in range(num_hour_period):
    for j in range(num_day_type):
        model.addConstr(x_blue[i, j] * mr73_car * num_cars[0] >= arr_demand_blue[i, j], 
                        name = "B demand " + time[j] + hour[i])
        model.addConstr(x_orange[i, j] * mpm10_car * num_cars[1] >= arr_demand_orange[i, j], 
                        name = "O demand " + time[j] + hour[i])
        model.addConstr(x_yellow[i, j] * mr73_car * num_cars[2] >= arr_demand_yellow[i, j], 
                        name = "Y demand " + time[j] + hour[i])
        model.addConstr(x_green[i, j] * (mr73_car + mpm10_car) * num_cars[3] >= arr_demand_green[i, j], 
                        name = "G demand " + time[j] + hour[i])

In [13]:
# 2. Passenger waiting time < Maximum waiting time
peak = [2, 3, 11, 12]
non_peak = []
for num in range(20):
    if num not in peak:
        non_peak.append(num)

for i in peak:
    model.addConstr(x_blue[i, 0] * mon_fri_peak[0] >= 60, name = "B weekday peak waiting")
    model.addConstr(x_orange[i, 0] * mon_fri_peak[1] >= 60, name = "O weekday peak waiting")
    model.addConstr(x_yellow[i, 0] * mon_fri_peak[2] >= 60, name = "Y weekday peak waiting")
    model.addConstr(x_green[i, 0] * mon_fri_peak[3] >= 60, name = "G weekday peak waiting")
    model.addConstr(x_blue[i, 1] * mon_fri_peak[0] >= 60, name = "B weekday peak waiting")
    model.addConstr(x_orange[i, 1] * mon_fri_peak[1] >= 60, name = "O weekday peak waiting")
    model.addConstr(x_yellow[i, 1] * mon_fri_peak[2] >= 60, name = "Y weekday peak waiting")
    model.addConstr(x_green[i, 1] * mon_fri_peak[3] >= 60, name = "G weekday peak waiting")
    
for i in non_peak:
    model.addConstr(x_blue[i, 0] * mon_fri_nonpeak[0] >= 60, name = "Blue weekday non-peak waiting")
    model.addConstr(x_orange[i, 0] * mon_fri_nonpeak[1] >= 60, name = "O weekday non-peak waiting")
    model.addConstr(x_yellow[i, 0] * mon_fri_nonpeak[2] >= 60, name = "Y weekday non-peak waiting")
    model.addConstr(x_green[i, 0] * mon_fri_nonpeak[3] >= 60, name = "G weekday non-peak waiting")
    model.addConstr(x_blue[i, 1] * mon_fri_nonpeak[0] >= 60, name = "Blue weekday non-peak waiting")
    model.addConstr(x_orange[i, 1] * mon_fri_nonpeak[1] >= 60, name = "O weekday non-peak waiting")
    model.addConstr(x_yellow[i, 1] * mon_fri_nonpeak[2] >= 60, name = "Y weekday non-peak waiting")
    model.addConstr(x_green[i, 1] * mon_fri_nonpeak[3] >= 60, name = "G weekday non-peak waiting")

for i in range(num_hour_period):
    for j in range(2,4):
        model.addConstr(x_blue[i, j] * sat_sun[0] >= 60, name = "B weekend waiting")
        model.addConstr(x_orange[i, j] * sat_sun[1] >= 60, name = "O weekend waiting")
        model.addConstr(x_yellow[i, j] * sat_sun[2] >= 60, name = "Y weekend waiting")
        model.addConstr(x_green[i, j] * sat_sun[3] >= 60, name = "G weekend waiting")

In [14]:
# 3. Budget constraint
model.addConstr(obj1 + obj2 + obj3 +  obj4 + fixed_cost <= budget, name = 'Budget constraint')

<gurobi.Constr *Awaiting Model Update*>

In [15]:
model.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 641 rows, 320 columns and 960 nonzeros
Model fingerprint: 0xac2c9488
Variable types: 0 continuous, 320 integer (0 binary)
Coefficient statistics:
  Matrix range     [2e+00, 6e+03]
  Objective range  [2e+02, 6e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+01, 5e+06]
Found heuristic solution: objective 6348534.0000
Presolve removed 641 rows and 320 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 6.34853e+06 

Optimal solution found (tolerance 1.00e-04)
Best objective 6.348534000000e+06, best bound 6.348534000000e+06, gap 0.0000%


In [16]:
model.objVal

6348534.0

In [17]:
for var in model.getVars():
    if var.x > 0:
        print(var.varName, '=', round(var.x, 2))

Blue #departure 5-6am_Mon-Thu = 6.0
Blue #departure 5-6am_Fri = 6.0
Blue #departure 5-6am_Sat = 6.0
Blue #departure 5-6am_Sun = 6.0
Blue #departure 6-7am_Mon-Thu = 10.0
Blue #departure 6-7am_Fri = 10.0
Blue #departure 6-7am_Sat = 6.0
Blue #departure 6-7am_Sun = 6.0
Blue #departure 7-8am_Mon-Thu = 12.0
Blue #departure 7-8am_Fri = 12.0
Blue #departure 7-8am_Sat = 7.0
Blue #departure 7-8am_Sun = 7.0
Blue #departure 8-9am_Mon-Thu = 12.0
Blue #departure 8-9am_Fri = 12.0
Blue #departure 8-9am_Sat = 8.0
Blue #departure 8-9am_Sun = 8.0
Blue #departure 9-10am_Mon-Thu = 7.0
Blue #departure 9-10am_Fri = 7.0
Blue #departure 9-10am_Sat = 10.0
Blue #departure 9-10am_Sun = 10.0
Blue #departure 10-11am_Mon-Thu = 7.0
Blue #departure 10-11am_Fri = 7.0
Blue #departure 10-11am_Sat = 7.0
Blue #departure 10-11am_Sun = 7.0
Blue #departure 11am-12pm_Mon-Thu = 7.0
Blue #departure 11am-12pm_Fri = 7.0
Blue #departure 11am-12pm_Sat = 7.0
Blue #departure 11am-12pm_Sun = 7.0
Blue #departure 12-13pm_Mon-Thu = 7.0
Bl

Green #departure 14-15pm_Sun = 5.0
Green #departure 15-16pm_Mon-Thu = 6.0
Green #departure 15-16pm_Fri = 7.0
Green #departure 15-16pm_Sat = 5.0
Green #departure 15-16pm_Sun = 5.0
Green #departure 16-17pm_Mon-Thu = 12.0
Green #departure 16-17pm_Fri = 12.0
Green #departure 16-17pm_Sat = 6.0
Green #departure 16-17pm_Sun = 6.0
Green #departure 17-18pm_Mon-Thu = 12.0
Green #departure 17-18pm_Fri = 12.0
Green #departure 17-18pm_Sat = 6.0
Green #departure 17-18pm_Sun = 6.0
Green #departure 18-19pm_Mon-Thu = 6.0
Green #departure 18-19pm_Fri = 7.0
Green #departure 18-19pm_Sat = 6.0
Green #departure 18-19pm_Sun = 5.0
Green #departure 19-20pm_Mon-Thu = 6.0
Green #departure 19-20pm_Fri = 6.0
Green #departure 19-20pm_Sat = 5.0
Green #departure 19-20pm_Sun = 5.0
Green #departure 20-21pm_Mon-Thu = 6.0
Green #departure 20-21pm_Fri = 6.0
Green #departure 20-21pm_Sat = 5.0
Green #departure 20-21pm_Sun = 5.0
Green #departure 21-22pm_Mon-Thu = 6.0
Green #departure 21-22pm_Fri = 6.0
Green #departure 21-22p

## Sensitivity analysis (Change maximum waiting time constraint)

In [18]:
for s in [0.5, 1, 1.5, 2]:
    # Create model
    model1 = gp.Model('STM minimizing cost model')

    # Create decision variables
    x_blue = model1.addVars(num_hour_period, num_day_type, vtype = GRB.INTEGER, 
                           name = ['Blue #departure ' + str(i) + '_' + str(j) 
                                   for i in hour for j in time])

    x_orange = model1.addVars(num_hour_period, num_day_type, vtype = GRB.INTEGER, 
                             name = ['Orange #departure ' + str(i) + '_' + str(j) 
                                   for i in hour for j in time])

    x_yellow = model1.addVars(num_hour_period, num_day_type, vtype = GRB.INTEGER, 
                             name = ['Yellow #departure ' + str(i) + '_' + str(j) 
                                   for i in hour for j in time])

    x_green = model1.addVars(num_hour_period, num_day_type, 
                            name = ['Green #departure ' + str(i) + '_' + str(j) 
                                   for i in hour for j in time])


    # Objective function
    # Blue
    obj1 = 4 * sum(x_blue[i, 0] * variable_cost[0]
                   for i in range(num_hour_period)) + sum(x_blue[i, j] * variable_cost[0]
                                                          for i in range(num_hour_period) 
                                                          for j in range(1, num_day_type))
    # Orange
    obj2 = 4 * sum(x_orange[i, 0] * variable_cost[1]
                   for i in range(num_hour_period)) + sum(x_orange[i, j] * variable_cost[1]
                                                          for i in range(num_hour_period) 
                                                          for j in range(1, num_day_type))

    # Yellow
    obj3 = 4 * sum(x_yellow[i, 0] * variable_cost[2]
                   for i in range(num_hour_period)) + sum(x_yellow[i, j] * variable_cost[2]
                                                          for i in range(num_hour_period) 
                                                          for j in range(1, num_day_type))

    # Green
    obj4 = 4 * sum(x_green[i, 0] * variable_cost[3]
                   for i in range(num_hour_period)) + sum(x_green[i, j] * variable_cost[3]
                                                          for i in range(num_hour_period) 
                                                          for j in range(1, num_day_type))
    # Set objective function
    model1.setObjective(obj1 + obj2 + obj3 +  obj4 + fixed_cost, GRB.MINIMIZE)


    # Constraints
    # 1. Capacity > Demand
    for i in range(num_hour_period):
        for j in range(num_day_type):
            model1.addConstr(x_blue[i, j] * mr73_car * num_cars[0] >= arr_demand_blue[i, j])
            model1.addConstr(x_orange[i, j] * mpm10_car * num_cars[1] >= arr_demand_orange[i, j])
            model1.addConstr(x_yellow[i, j] * mr73_car * num_cars[2] >= arr_demand_yellow[i, j])
            model1.addConstr(x_green[i, j] * (mr73_car + mpm10_car) * num_cars[3] >= arr_demand_green[i, j])

    # 2. Passenger waiting time < Maximum waiting time
    peak = [2, 3, 11, 12]
    non_peak = []
    for num in range(20):
        if num not in peak:
            non_peak.append(num)

    for i in peak:
        model1.addConstr(x_blue[i, 0] * (mon_fri_peak[0] + s) >= 60)
        model1.addConstr(x_orange[i, 0] * (mon_fri_peak[1] + s) >= 60)
        model1.addConstr(x_yellow[i, 0] * (mon_fri_peak[2] + s) >= 60)
        model1.addConstr(x_green[i, 0] * (mon_fri_peak[3] + s) >= 60)
        model1.addConstr(x_blue[i, 1] * (mon_fri_peak[0] + s) >= 60)
        model1.addConstr(x_orange[i, 1] * (mon_fri_peak[1] + s) >= 60)
        model1.addConstr(x_yellow[i, 1] * (mon_fri_peak[2] + s) >= 60)
        model1.addConstr(x_green[i, 1] * (mon_fri_peak[3] + s) >= 60)

    for i in non_peak:
        model1.addConstr(x_blue[i, 0] * (mon_fri_nonpeak[0] + s) >= 60)
        model1.addConstr(x_orange[i, 0] * (mon_fri_nonpeak[1] + s) >= 60)
        model1.addConstr(x_yellow[i, 0] * (mon_fri_nonpeak[2] + s) >= 60)
        model1.addConstr(x_green[i, 0] * (mon_fri_nonpeak[3] + s) >= 60)
        model1.addConstr(x_blue[i, 1] * (mon_fri_nonpeak[0] + s) >= 60)
        model1.addConstr(x_orange[i, 1] * (mon_fri_nonpeak[1] + s) >= 60)
        model1.addConstr(x_yellow[i, 1] * (mon_fri_nonpeak[2] + s) >= 60)
        model1.addConstr(x_green[i, 1] * (mon_fri_nonpeak[3] + s) >= 60)

    for i in range(num_hour_period):
        for j in range(2,4):
            model1.addConstr(x_blue[i, j] * (sat_sun[0] + s) >= 60)
            model1.addConstr(x_orange[i, j] * (sat_sun[1] + s) >= 60)
            model1.addConstr(x_yellow[i, j] * (sat_sun[2] + s) >= 60)
            model1.addConstr(x_green[i, j] * (sat_sun[3] + s) >= 60)

    model1.optimize()
    print("---------------------------------------------------------------------------------------------------")
    print("Sensitivity Analysis of changing maximum waiting time constraint:\n")
    print("When we change the average waiting time by ", s, 
          "minute (for all non-peak, peak, and weekend periods), the optimal cost is ", model1.objVal)
    print("---------------------------------------------------------------------------------------------------")

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 640 rows, 320 columns and 640 nonzeros
Model fingerprint: 0xb392cfaa
Variable types: 80 continuous, 240 integer (0 binary)
Coefficient statistics:
  Matrix range     [3e+00, 3e+03]
  Objective range  [2e+02, 6e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+01, 2e+04]
Found heuristic solution: objective 1.067080e+15
Presolve removed 640 rows and 320 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 2: 6.13066e+06 1.06708e+15 

Optimal solution found (tolerance 1.00e-04)
Best objective 6.130660232979e+06, best bound 6.130660232979e+06, gap 0.0000%
---------------------------------------------------------------------------------------------------
Sensitivity Analysis of changing maximum w